In [12]:
import numpy as np
import cvxopt #Linear and Integer Programming Solver
import cvxopt.glpk
from cvxopt import matrix
import matplotlib.pyplot as plt

In [13]:
#We have in total 88 binary variables
nbr_bricks = 22 #Defining number of bricks
nbr_SRs = 5 #Defining number of sales representatives
#The brick id goes from 1 to 22
#The sales representative id goes from 1 to 5

In [14]:
#Creates a unique ID for each variable
def get_id(i,j):
    '''
    Associate a unique variable index given a 2-index (i,j) 
    '''
    assert(i>=0 and i<nbr_bricks)
    assert(j>=0 and j<nbr_SRs)
    
    return(5*i + j)

In [15]:
def printsol(sol):
    header = " "*10 + "SR1 SR2 SR3 SR4 SR5"
    print(header)
    sep = " "*9 + "+---+---+---+---+---+"
    print(sep)
    for i in range(nbr_bricks):
        if i+1<10:
            print("Brick {}  ".format(i+1),end='')
        else:
            print("Brick {} ".format(i+1),end='')
        for j in range(nbr_SRs):
            print(" ",end='')
            if (sol[get_id(i,j)]==1):
                print(" X ",end='')
            else:
                print("   ",end='')
   
        print("")
        print(sep)

In [16]:
#Function to normalize data to the range [0,1]
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [17]:
#Function to compute best solution for the multicriteria problem based on SR5 center brick
def compute_sol_center_brick(SR5_center_brick):
    #Defining constant values
    #Brick Index Value Array:
    brick_index_prev = np.array([0.1609, 0.1164, 0.1026, 0.1516, 0.0939, 0.1320, 0.0687, 0.0930, 0.2116, 0.2529, 0.0868, 0.0828, 0.0975, 0.8177, 0.4115, 0.3795, 0.0710, 0.0427, 0.1043, 0.0997, 0.1698, 0.2531])
    brick_index = 1.20*brick_index_prev #Increased Demand
    #Travel Distance between SR offices and bricks Matrix:
    distances_all_bricks = np.array([[0.00,7.35,13.21,16.16,18.22,16.68,14.00,19.02,17.45,28.49,27.20,29.30,24.53,24.09,25.38,24.29,23.23,23.70,8.48,13.85,5.24,21.12] ,[7.35,0.00,6.58,19.00,20.06,19.06,16.30,21.59,21.45,30.99,29.89,25.60,25.98,26.47,28.22,27.22,26.25,26.27,8.08,8.36,9.15,17.33] ,[13.21,6.58,0.00,25.29,26.05,25.24,22.54,27.76,27.93,36.99,35.97,27.90,31.68,32.50,34.37,33.41,32.47,32.36,14.18,10.96,15.72,12.25] ,[16.16,19.00,25.29,0.00,3.07,1.22,2.80,2.87,3.80,12.35,11.11,21.99,8.82,7.94,9.34,8.29,7.28,7.55,11.13,17.49,11.03,36.12] ,[18.22,20.06,26.05,3.07,0.00,1.92,4.22,2.35,6.19,10.94,9.93,19.37,6.31,6.45,8.40,7.55,6.77,6.35,11.98,17.22,12.99,37.37] ,[16.68,19.06,25.24,1.22,1.92,0.00,2.79,2.52,4.83,12.01,10.85,20.81,7.98,7.51,9.16,8.17,7.23,7.23,11.07,17.01,11.48,36.29] ,[14.00,16.30,22.54,2.80,4.22,2.79,0.00,5.30,6.20,14.80,13.64,20.87,10.53,10.30,11.93,10.92,9.96,10.02,8.36,14.77,8.78,33.50] ,[19.02,21.59,27.76,2.87,2.35,2.52,5.30,0.00,4.42,9.53,8.34,21.46,6.14,5.07,6.63,5.65,4.73,4.74,13.59,19.33,13.87,38.80] ,[17.45,21.45,27.93,3.80,6.19,4.83,6.20,4.42,0.00,11.74,10.33,25.54,10.15,8.02,8.47,7.37,6.31,7.36,14.00,20.92,12.73,38.16] ,[28.49,30.99,36.99,12.35,10.94,12.01,14.80,9.53,11.74,0.00,1.45,24.31,6.06,4.51,3.27,4.37,5.44,4.80,22.92,27.74,23.38,48.27] ,[27.20,29.89,35.97,11.11,9.93,10.85,13.64,8.34,10.33,1.45,0.00,24.44,5.75,3.51,1.86,2.97,4.04,3.62,21.85,26.94,22.12,47.14] ,[29.30,25.60,27.90,21.99,19.37,20.81,20.87,21.46,25.54,24.31,24.44,0.00,18.86,22.01,24.20,24.10,24.02,22.58,20.93,17.43,25.40,39.86] ,[24.53,25.98,31.68,8.82,6.31,7.98,10.53,6.14,10.15,6.06,5.75,18.86,0.00,3.28,5.34,5.38,5.60,3.98,17.93,21.98,19.29,43.31] ,[24.09,26.47,32.50,7.94,6.45,7.51,10.30,5.07,8.02,4.51,3.51,22.01,3.28,0.00,2.32,2.10,2.40,0.77,18.41,23.44,18.94,43.75] ,[25.38,28.22,34.37,9.34,8.40,9.16,11.93,6.63,8.47,3.27,1.86,24.20,5.34,2.32,0.00,1.11,2.19,2.09,20.21,25.57,20.33,45.43] ,[24.29,27.22,33.41,8.29,7.55,8.17,10.92,5.65,7.37,4.37,2.97,24.10,5.38,2.10,1.11,0.00,1.08,1.54,19.24,24.76,19.26,44.40] ,[23.23,26.25,32.47,7.28,6.77,7.23,9.96,4.73,6.31,5.44,4.04,24.02,5.60,2.40,2.19,1.08,0.00,1.64,18.29,23.98,18.21,43.40] ,[23.70,26.27,32.36,7.55,6.35,7.23,10.02,4.74,7.36,4.80,3.62,22.58,3.98,0.77,2.09,1.54,1.64,0.00,18.23,23.48,18.58,43.52] ,[8.48,8.08,14.18,11.13,11.98,11.07,8.36,13.59,14.00,22.92,21.85,20.93,17.93,18.41,20.21,19.24,18.29,18.23,0.00,7.85,4.79,25.40] ,[13.85,8.36,10.96,17.49,17.22,17.01,14.77,19.33,20.92,27.74,26.94,17.43,21.98,23.44,25.57,24.76,23.98,23.48,7.85,0.00,12.23,23.21] ,[5.24,9.15,15.72,11.03,12.99,11.48,8.78,13.87,12.73,23.38,22.12,25.40,19.29,18.94,20.33,19.26,18.21,18.58,4.79,12.23,0.00,25.43] ,[21.12,17.33,12.25,36.12,37.37,36.29,33.50,38.80,38.16,48.27,47.14,39.86,43.31,43.75,45.43,44.40,43.40,43.52,25.40,23.21,25.43,0.00]])
    distances_prev = np.array([[16.16, 24.08, 24.32, 21.12] ,[19, 26.47, 27.24, 17.33] ,[25.29, 32.49, 33.42, 12.25] ,[0, 7.93, 8.31, 36.12] ,[3.07, 6.44, 7.56, 37.37] ,[1.22, 7.51, 8.19, 36.29] ,[2.8, 10.31, 10.95, 33.5] ,[2.87, 5.07, 5.67, 38.8] ,[3.8, 8.01, 7.41, 38.16] ,[12.35, 4.52, 4.35, 48.27] ,[11.11, 3.48, 2.97, 47.14] ,[21.99, 22.02, 24.07, 39.86] ,[8.82, 3.3, 5.36, 43.31] ,[7.93, 0, 2.07, 43.75] ,[9.34, 2.25, 1.11, 45.43] ,[8.31, 2.07, 0, 44.43] ,[7.31, 2.44, 1.11, 43.43] ,[7.55, 0.75, 1.53, 43.52] ,[11.13, 18.41, 19.26, 25.4] ,[17.49, 23.44, 24.76, 23.21] ,[11.03, 18.93, 19.28, 25.43] ,[36.12, 43.75, 44.43, 0]])
    distances = np.concatenate((distances_prev,distances_all_bricks[:,SR5_center_brick-1].reshape(-1,1)),axis=1)

    #Let's define the equality matrix
    #We have in total 27 equality constraints
    A = np.zeros((27,nbr_bricks*nbr_SRs))
    c=0 #Starting the constraint counter

    #Defining sales representatives offices (center bricks): 5 constraints
    for i in range(nbr_bricks):
        for j in range(nbr_SRs):
            #Define the office of Sales Representative 1 at brick 4
            if i+1==4 and j+1==1: 
                A[c,get_id(i,j)]=1
                c+=1
            #Define the office of Sales Representative 2 at brick 14
            if i+1==14 and j+1==2:
                A[c,get_id(i,j)]=1
                c+=1
            #Define the office of Sales Representative 3 at brick 16
            if i+1==16 and j+1==3:
                A[c,get_id(i,j)]=1
                c+=1
            #Define the office of Sales Representative 4 at brick 22
            if i+1==22 and j+1==4:
                A[c,get_id(i,j)]=1
                c+=1
            #Define the office of Sales Representative 5 at brick 1
            if i+1==1 and j+1==5:
                A[c,get_id(i,j)]=1
                c+=1            

    #Now we add the constraint that each brick cannot be assign to more than 1 sales representative at a time: 22 constraints
    for i in range(nbr_bricks):
        for j in range(nbr_SRs):
            A[c,get_id(i,j)]=1 #Each brick can only be assigned to one SR at a time
        c+=1

    #Now let's define the inequality matrix
    #We have in total 10 inequality constraints
    G = np.zeros((10,nbr_bricks*nbr_SRs))
    c=0 #Starting the constraint counter

    #We also need to create the h array that will represent all values on the right-hand side of the inequalities
    h=np.zeros(10)

    #Defining the workload threshold [0.8-1.2]
    workload_l = 0.8
    workload_h = 1.2

    #Defining workload upper limit
    for j in range(nbr_SRs):
        for i in range(nbr_bricks):
            G[c,get_id(i,j)]=brick_index[i]
            h[c]=workload_h
        c+=1

    #Defining workload lower limit    
    for j in range(nbr_SRs):
        for i in range(nbr_bricks):
            G[c,get_id(i,j)]=-brick_index[i]
            h[c]=-workload_l
        c+=1        

    G.shape

    #We initialize our distance travelled by SR'S
    total_distance = np.zeros(nbr_bricks*nbr_SRs)

    #Defining the distance objective
    for i in range(nbr_bricks):
        for j in range(nbr_SRs):
            total_distance[get_id(i,j)] = distances[i][j]

    #Initializing disruption matrix
    total_disruption = np.zeros(nbr_bricks*nbr_SRs)

    #Defining the disruption objective
    for i in range(nbr_bricks):
        for j in range(nbr_SRs):
            if (i+1==1 and j+1==4) or (i+1==2 and j+1==4) or (i+1==3 and j+1==4) or (i+1==5 and j+1==1) or (i+1==6 and j+1==1) \
            or (i+1==7 and j+1==1) or (i+1==8 and j+1==1) or (i+1==9 and j+1==3) or (i+1==10 and j+1==2) or (i+1==11 and j+1==2) \
            or (i+1==12 and j+1==2) or (i+1==13 and j+1==2) or (i+1==15 and j+1==1) or (i+1==17 and j+1==3) \
            or (i+1==18 and j+1==3) or (i+1==19 and j+1==4) or (i+1==20 and j+1==4) or (i+1==21 and j+1==4):
                total_disruption[get_id(i,j)]=-brick_index[i]

    #We will first minimize the total disruption and then start adding the total distance as a new constraint in order to implement the epsilon-constraint algorithm

    Aeq = matrix(A)
    b=matrix(np.ones(A.shape[0]))
    G=matrix(G)
    h=matrix(h)
    cost_disrup=matrix(total_disruption) #total disruption as cost function
    cost_distance = matrix(total_distance) #distance objective
    B=set(range(A.shape[1])) #Set of binary variables
    #We use a factor of 0.00001 in order to remove weekly non-dominated solutions
    (status, solution) = cvxopt.glpk.ilp(c=cost_disrup+0.00001*cost_distance,G=G,h=h,A=Aeq,b=b,B=B)

    disruption_partial = cost_disrup.T*solution

    #Completing disruption formula
    sum_disrupt_base=0
    for i in range(nbr_bricks):
        if i+1 not in (4,14,16,22):
            sum_disrupt_base +=  brick_index[i]

    #Computing the disruption value for the given conditions
    disruption_final = disruption_partial[0] + sum_disrupt_base

    #We initialize our distance travelled by SR'S (distance_final) for the first time before entering the iteration 
    #process because the first time the algorithm does not have the value beforehand
    distance_final = cost_distance.T*solution

    ### Now in order to implement the epsilon-method we need to start iterating while adding a new constraint based on the second objective function (Total distance)

    #Initializing distance and disruption solution points
    distance_final_points = []
    disruption_final_points = []
    #Initializing solver solutions
    solution_list = []

    disruption_final_points.append(disruption_final)
    distance_final_points.append(distance_final[0])
    solution_list.append(solution)

    nbr_iter = 1000 #Max number of iterations

    for k in range(nbr_iter):

        #Defining the inequality matrix
        G = np.zeros((11,nbr_bricks*nbr_SRs))
        c=0 #Starting the constraint counter
        #Initializing h array that will represent all values on the right-hand side of the inequalities
        h=np.zeros(11)   

        #Balanced Workload upper limit
        for j in range(nbr_SRs):
            for i in range(nbr_bricks):
                G[c,get_id(i,j)]=brick_index[i]
                h[c]=workload_h
            c+=1

        #Balanced Workload lower limit
        for j in range(nbr_SRs):
            for i in range(nbr_bricks):
                G[c,get_id(i,j)]=-brick_index[i]
                h[c]=-workload_l
            c+=1   

        #Epsilon Constraint
        for i in range(nbr_bricks):
            for j in range(nbr_SRs):
                G[c,get_id(i,j)] = distances[i][j]
        h[c]=distance_final[0]-0.001 #Epsilon = 0.001       
        c+=1

        #Minimizing Disruption while having Distance as a constraint
        Aeq = matrix(A)
        b=matrix(np.ones(A.shape[0]))
        G=matrix(G)
        h=matrix(h)
        cost_disrup=matrix(total_disruption) #total disruption as cost function
        B=set(range(A.shape[1])) #Set of binary variables
        #We use a factor of 0.00001 in order to remove weekly non-dominated solutions
        (status, solution) = cvxopt.glpk.ilp(c=cost_disrup+0.00001*cost_distance,G=G,h=h,A=Aeq,b=b,B=B)

        #Breaks when no more optimal solutions found
        #print("Status for iteration {}: {}".format(k+1,status))
        if status != 'optimal':
            break

        solution_list.append(solution) #Saving the current solution in a list

        #Disruption
        disruption_partial = cost_disrup.T*solution    
        sum_disrupt_base=0
        for i in range(nbr_bricks):
            if i+1 not in (4,14,16,22):
                sum_disrupt_base +=  brick_index[i]    
        #Computing the disruption value for the given conditions
        disruption_final = disruption_partial[0] + sum_disrupt_base    

        #Distance
        #For this particular condition let's compute the distance travelled by SRs    
        total_distance = np.zeros(nbr_bricks*nbr_SRs)

        for i in range(nbr_bricks):
            for j in range(nbr_SRs):
                total_distance[get_id(i,j)] = distances[i][j]    

        cost_distance = matrix(total_distance)
        distance_final = cost_distance.T*solution #This value will be used in the next iteration in the epsilon constraint

        #Storing objective values
        disruption_final_points.append(disruption_final)
        distance_final_points.append(distance_final[0])  



    #Zipping solution points
    non_dominated_solutions = list(zip([round(num,2) for num in disruption_final_points], [round(num,2) for num in distance_final_points]))

    #Using an additive aggregation function 
    #with weights = 0.5 for both criteria Disruption and Distance
    additive_values = []

    for i in range(len(disruption_final_points)):
        weighted_value = 0.5*NormalizeData(disruption_final_points)[i] + 0.5*NormalizeData(distance_final_points)[i]
        additive_values.append(weighted_value)

    #Best aggregated solution disruption and distance values
    #non_dominated_solutions[np.argmin(additive_values)]

    return non_dominated_solutions[np.argmin(additive_values)], solution_list[np.argmin(additive_values)]

In [18]:
#Defining Center Brick of SR5 (from 1 to 22) in each iteration and computing best solution based on additive model
for i in range(nbr_bricks):
    solution_point, solution_matrix = compute_sol_center_brick(i+1)
    print("Best Solution for SR5 at Center Brick {}: {}".format(i+1,solution_point))

Best Solution for SR5 at Center Brick 1: (0.82, 191.3)
Best Solution for SR5 at Center Brick 2: (1.21, 165.29)
Best Solution for SR5 at Center Brick 3: (1.21, 182.15)
Best Solution for SR5 at Center Brick 4: (0.91, 175.47)
Best Solution for SR5 at Center Brick 5: (0.94, 171.28)
Best Solution for SR5 at Center Brick 6: (0.94, 173.26)
Best Solution for SR5 at Center Brick 7: (0.91, 169.07)
Best Solution for SR5 at Center Brick 8: (0.94, 174.57)
Best Solution for SR5 at Center Brick 9: (0.8, 185.11)
Best Solution for SR5 at Center Brick 10: (1.3, 167.76)
Best Solution for SR5 at Center Brick 11: (1.3, 165.06)
Best Solution for SR5 at Center Brick 12: (0.83, 203.14)
Best Solution for SR5 at Center Brick 13: (0.84, 178.43)
Best Solution for SR5 at Center Brick 14: (0.94, 178.32)
Best Solution for SR5 at Center Brick 15: (0.8, 183.7)
Best Solution for SR5 at Center Brick 16: (0.8, 183.75)
Best Solution for SR5 at Center Brick 17: (0.8, 183.78)
Best Solution for SR5 at Center Brick 18: (0.8, 

In [19]:
#Based on these results
#The Best Solution would be to put SR5 at Center Brick 7

In [22]:
solution_point, solution_matrix = compute_sol_center_brick(7)
print("Best Solution for SR5 at Center Brick {}: {}".format(7,solution_point))

Best Solution for SR5 at Center Brick 7: (0.91, 169.07)


## Printing Solution Minimum Disruption and Distance

In [23]:
printsol(solution_matrix)

          SR1 SR2 SR3 SR4 SR5
         +---+---+---+---+---+
Brick 1                    X 
         +---+---+---+---+---+
Brick 2                X     
         +---+---+---+---+---+
Brick 3                X     
         +---+---+---+---+---+
Brick 4    X                 
         +---+---+---+---+---+
Brick 5    X                 
         +---+---+---+---+---+
Brick 6    X                 
         +---+---+---+---+---+
Brick 7                    X 
         +---+---+---+---+---+
Brick 8    X                 
         +---+---+---+---+---+
Brick 9            X         
         +---+---+---+---+---+
Brick 10                   X 
         +---+---+---+---+---+
Brick 11           X         
         +---+---+---+---+---+
Brick 12                   X 
         +---+---+---+---+---+
Brick 13       X             
         +---+---+---+---+---+
Brick 14       X             
         +---+---+---+---+---+
Brick 15   X                 
         +---+---+---+---+---+
Brick 16           X    